In [112]:
import pandas as pd
import numpy as np
from IPython.display import display
import statistics
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [40]:
df = pd.read_csv('maxreturngivenrisk.csv')
pd.options.display.max_columns = None
display(df)

,Price Matrix,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,Timestamp(months),Stock 1,Stock 2,Stock 3,Stock 4,Stock 5,Stock 6,Stock 7,Stock 8,Stock 9,Stock 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,30,35,100,150,10,25,75,600,250,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,29.40,35.00,101.00,144.00,10.10,25.75,75.75,648.00,260.00,273.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,30.28,33.95,102.01,141.12,10.40,26.01,78.02,686.88,241.80,281.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,28.47,34.63,104.05,141.12,10.61,26.79,80.36,734.96,246.64,292.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,28.75,36.71,104.05,141.12,10.82,27.32,81.97,764.36,261.43,286.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,29.90,38.54,105.09,141.12,10.82,27.87,84.43,817.87,269.28,292.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6,30.50,38.16,105.09,135.48,11.04,28.71,87.81,858.76,261.20,277.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,30.19,40.45,107.19,131.41,11.26,29.28,92.20,944.63,245.53,308.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8,31.10,40.04,107.19,128.78,11.37,30.16,91.28,982.42,230.80,360.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
s_prices = df.iloc[1:14,1:11]

In [52]:
s_prices

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,30,35,100,150,10,25,75,600,250,300
2,29.40,35.00,101.00,144.00,10.10,25.75,75.75,648.00,260.00,273.00
3,30.28,33.95,102.01,141.12,10.40,26.01,78.02,686.88,241.80,281.19
4,28.47,34.63,104.05,141.12,10.61,26.79,80.36,734.96,246.64,292.44
5,28.75,36.71,104.05,141.12,10.82,27.32,81.97,764.36,261.43,286.59
6,29.90,38.54,105.09,141.12,10.82,27.87,84.43,817.87,269.28,292.32
7,30.50,38.16,105.09,135.48,11.04,28.71,87.81,858.76,261.20,277.70
8,30.19,40.45,107.19,131.41,11.26,29.28,92.20,944.63,245.53,308.25
9,31.10,40.04,107.19,128.78,11.37,30.16,91.28,982.42,230.80,360.65
10,29.85,42.44,109.34,127.49,11.71,30.46,90.36,1011.89,244.64,364.26


In [69]:
returns= pd.DataFrame(columns= s_prices.columns, index= s_prices.index)
print(returns)

   Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
1         NaN        NaN        NaN        NaN        NaN        NaN   
2         NaN        NaN        NaN        NaN        NaN        NaN   
3         NaN        NaN        NaN        NaN        NaN        NaN   
4         NaN        NaN        NaN        NaN        NaN        NaN   
5         NaN        NaN        NaN        NaN        NaN        NaN   
6         NaN        NaN        NaN        NaN        NaN        NaN   
7         NaN        NaN        NaN        NaN        NaN        NaN   
8         NaN        NaN        NaN        NaN        NaN        NaN   
9         NaN        NaN        NaN        NaN        NaN        NaN   
10        NaN        NaN        NaN        NaN        NaN        NaN   
11        NaN        NaN        NaN        NaN        NaN        NaN   
12        NaN        NaN        NaN        NaN        NaN        NaN   
13        NaN        NaN        NaN        NaN        NaN       

In [72]:
for i in range(1,13):
    for j in range(0,10):
        returns.iloc[i,j]= (float(s_prices.iloc[i,j]) - float(s_prices.iloc[i-1,j]))/float(s_prices.iloc[i-1,j])
returns

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.02,0.0,0.01,-0.04,0.01,0.03,0.01,0.08,0.04,-0.09
3,0.029932,-0.03,0.01,-0.02,0.029703,0.010097,0.029967,0.06,-0.07,0.03
4,-0.059775,0.020029,0.019998,0.0,0.020192,0.029988,0.029992,0.069998,0.020017,0.040009
5,0.009835,0.060064,0.0,0.0,0.019793,0.019784,0.020035,0.040002,0.059966,-0.020004
6,0.04,0.04985,0.009995,0.0,0.0,0.020132,0.030011,0.070006,0.030027,0.019994
7,0.020067,-0.00986,0.0,-0.039966,0.020333,0.03014,0.040033,0.049996,-0.030006,-0.050014
8,-0.010164,0.06001,0.019983,-0.030041,0.019928,0.019854,0.049994,0.099993,-0.059992,0.110011
9,0.030142,-0.010136,0.0,-0.020014,0.009769,0.030055,-0.009978,0.040005,-0.059993,0.169992
10,-0.040193,0.05994,0.020058,-0.010017,0.029903,0.009947,-0.010079,0.029997,0.059965,0.01001


In [104]:
returns = returns.astype('float64')
covmat = returns.cov()
covmat

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
Unnamed: 1,0.001208,-0.000110,-0.000136,-0.000008,-1.314484e-04,-0.000061,8.240907e-05,-0.000472,-0.000365,0.000619
Unnamed: 2,-0.000110,0.001017,0.000113,0.000258,-3.312120e-05,-0.000104,8.879132e-05,-0.000200,0.000660,0.000004
Unnamed: 3,-0.000136,0.000113,0.000058,0.000023,1.643897e-05,-0.000021,3.235536e-05,0.000029,0.000004,0.000006
Unnamed: 4,-0.000008,0.000258,0.000023,0.000297,-2.889790e-05,-0.000079,-1.707902e-05,-0.000162,0.000308,0.000157
Unnamed: 5,-0.000131,-0.000033,0.000016,-0.000029,7.152969e-05,-0.000016,1.864724e-07,0.000010,-0.000075,-0.000068
Unnamed: 6,-0.000061,-0.000104,-0.000021,-0.000079,-1.619111e-05,0.000070,-4.489795e-06,0.000095,-0.000041,-0.000029
Unnamed: 7,0.000082,0.000089,0.000032,-0.000017,1.864724e-07,-0.000004,3.246519e-04,0.000136,-0.000320,-0.000209
Unnamed: 8,-0.000472,-0.000200,0.000029,-0.000162,1.028811e-05,0.000095,1.363035e-04,0.000817,-0.000289,-0.000142
Unnamed: 9,-0.000365,0.000660,0.000004,0.000308,-7.484742e-05,-0.000041,-3.197318e-04,-0.000289,0.002245,-0.001268
Unnamed: 10,0.000619,0.000004,0.000006,0.000157,-6.806588e-05,-0.000029,-2.091489e-04,-0.000142,-0.001268,0.004703


In [91]:
returns.loc[13]=[None, None, None, None, None, None, None, None, None, None]
returns
returns.loc[14]=[None, None, None, None, None, None, None, None, None, None]
returns

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.02,0.0,0.01,-0.04,0.01,0.03,0.01,0.08,0.04,-0.09
3,0.029932,-0.03,0.01,-0.02,0.029703,0.010097,0.029967,0.06,-0.07,0.03
4,-0.059775,0.020029,0.019998,0.0,0.020192,0.029988,0.029992,0.069998,0.020017,0.040009
5,0.009835,0.060064,0.0,0.0,0.019793,0.019784,0.020035,0.040002,0.059966,-0.020004
6,0.04,0.04985,0.009995,0.0,0.0,0.020132,0.030011,0.070006,0.030027,0.019994
7,0.020067,-0.00986,0.0,-0.039966,0.020333,0.03014,0.040033,0.049996,-0.030006,-0.050014
8,-0.010164,0.06001,0.019983,-0.030041,0.019928,0.019854,0.049994,0.099993,-0.059992,0.110011
9,0.030142,-0.010136,0.0,-0.020014,0.009769,0.030055,-0.009978,0.040005,-0.059993,0.169992
10,-0.040193,0.05994,0.020058,-0.010017,0.029903,0.009947,-0.010079,0.029997,0.059965,0.01001


In [90]:
er= np.nanmean(returns, axis=0)       #for expected returns of each stock
stddev= np.nanstd(returns, axis=0)    #for standard dev of each stock
print(er)
print(stddev)

[-0.00727998  0.01726992  0.00909117 -0.01727284  0.01819996  0.02272316
  0.01909149  0.06909069  0.00090868  0.0236373 ]
[0.03292297 0.03163885 0.00759218 0.01476646 0.00796712 0.00718785
 0.01801563 0.02596545 0.04571931 0.06749008]


In [92]:
returns.iloc[12]= er       #appends expected return of each stock at row 13
returns.iloc[13]= stddev   #appends std dev of each stock at row 14
returns

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.02,0.0,0.01,-0.04,0.01,0.03,0.01,0.08,0.04,-0.09
3,0.029932,-0.03,0.01,-0.02,0.029703,0.010097,0.029967,0.06,-0.07,0.03
4,-0.059775,0.020029,0.019998,0.0,0.020192,0.029988,0.029992,0.069998,0.020017,0.040009
5,0.009835,0.060064,0.0,0.0,0.019793,0.019784,0.020035,0.040002,0.059966,-0.020004
6,0.04,0.04985,0.009995,0.0,0.0,0.020132,0.030011,0.070006,0.030027,0.019994
7,0.020067,-0.00986,0.0,-0.039966,0.020333,0.03014,0.040033,0.049996,-0.030006,-0.050014
8,-0.010164,0.06001,0.019983,-0.030041,0.019928,0.019854,0.049994,0.099993,-0.059992,0.110011
9,0.030142,-0.010136,0.0,-0.020014,0.009769,0.030055,-0.009978,0.040005,-0.059993,0.169992
10,-0.040193,0.05994,0.020058,-0.010017,0.029903,0.009947,-0.010079,0.029997,0.059965,0.01001


In [101]:
data = [[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1]]
WT = pd.DataFrame(data) #assigning random weights to stocks (transpose)
WT

,0
0,0.1
1,0.1
2,0.1
3,0.1
4,0.1
5,0.1
6,0.1
7,0.1
8,0.1
9,0.1


In [102]:
W= np.transpose(WT)      #transpose of WT (W)
W

,0,1,2,3,4,5,6,7,8,9
0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [109]:
portfolio_var = np.matmul(W, np.matmul(covmat, WT))  #W*CovarianceMatrix*WTranspose
portfolio_sd = np.sqrt(portfolio_var)
portfolio_sd

<ipython-input-109-23f094cb9f86>:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  portfolio_var = np.matmul(W, np.matmul(covmat, WT))  #W*CovarianceMatrix*WTranspose


,0
0,0.008556


In [169]:
portfolio_return = np.matmul(er, WT)       #generating portfolio return by multiplying weights with expected returns on each stock
portfolio_return


0    0.015546
dtype: float64

In [126]:
#model = pyo.ConcreteModel() 
#variables
##model.w = Var(W, bounds=(0,1))
#model.port_std = Var(portfolio_sd)
#model.port_ret = Var(portfolio_return)


In [127]:
#constraints Case 1: maximizing return for given risk, for max risk is 4%
#model.limits = ConstraintList()
#model.C1= Constraint(model.w['i'] for i in model.w <= 1.0)
#model.C2= Constraint(model.w['i'] for i in model.w >= 0)
#model.C3= Constraint(model.port_std <= .04)


TypeError: '<=' not supported between instances of 'IndexedVar' and 'float'

In [124]:
#objective
#model.O = Objective(expr = port_ret, sense= maximize)


0    0.1
Name: 7, dtype: float64

In [163]:
#variables
model=pyo.ConcreteModel()
model.port= pyo.Var(range(10),domain=NonNegativeReals)
port= model.port



In [164]:
#constraints
model.C1 = pyo.Constraint(expr = sum(port[i] for i in range(10)) <=1.0)
model.C2 = pyo.Constraint(expr = np.matmul(port, np.matmul(covmat.to_numpy(), np.transpose(port)) <= .04)


SyntaxError: unexpected EOF while parsing (<ipython-input-164-89ff9bccb219>, line 3)

In [171]:
#objective (to maximize return for given risk)
port_return = np.matmul(er,np.transpose(port))
model.O = pyo.Objective(expr=port_return, sense= maximize)

    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [172]:
opt = SolverFactory('ipopt')
opt.solve(model)

    ipopt


ApplicationError: No executable found for solver 'ipopt'